## Constraint detecting model

In this notebook the model to detect constraints is trained.

In previous notebooks the data for this process has been generated.
In this notebook the architecture of the model is defined.
The model is then trained to check if the results are good enough to proceed.

First the prepared data has to be loaded.

The dataset is split into three parts.
The training set containts 100.000 images, the validation set 20.000 and the test set contains the rest of the 119.190 images (19.190).

`src.utils.decode_image_record` can not be used here, because it needs the data to be split before calling it.
Thoughts on the improvement to the data generation process can be read in the conclusion to this first prototype in the report-document.

In [1]:
from datetime import datetime
from os import path

import tensorflow as tf

In [2]:
record_path = path.join('data', 'sep_processed_01', 'crops.tfrecord')

feature_description = {
    'image': tf.io.FixedLenFeature([], tf.string),
    'label': tf.io.FixedLenFeature([], tf.int64)
}

def decode(example):
    feature = tf.io.parse_single_example(example, feature_description)
    image = tf.io.parse_tensor(feature['image'], tf.float32)
    image.set_shape((96, 96, 1))
    label = feature['label']
    return [image, label]

dataset = tf.data.TFRecordDataset(record_path).shuffle(1000)

def sanitize_dataset(ds):
    autotune = tf.data.experimental.AUTOTUNE
    return (ds
        .map(decode, num_parallel_calls=autotune)    
        .cache()
        .repeat()
        .batch(32)
        .prefetch(buffer_size=autotune))

train = sanitize_dataset(dataset.take(80000))
validate = sanitize_dataset(dataset.skip(80000).take(20000))
test = sanitize_dataset(dataset.skip(100000))

As a baseline the model trained in the `srp` is used.
The only difference is that the input layer expects 96 by 96 images.

This time the model is created using the functional API, which holds no functional differences here.

In [3]:
inputs = tf.keras.Input(shape=(96, 96, 1))
m = tf.keras.layers.Conv2D(16, (4,4), activation='relu', padding='same')(inputs)
m = tf.keras.layers.MaxPooling2D(2,2)(m)
m = tf.keras.layers.Conv2D(32, (4,4), activation='relu', padding='same')(m)
m = tf.keras.layers.MaxPooling2D(2,2)(m)
m = tf.keras.layers.Flatten()(m)
outputs = tf.keras.layers.Dense(3, 'softmax')(m)

model = tf.keras.Model(inputs = inputs, outputs = outputs)

log_dir = path.join('logs', 'sep231', datetime.now().strftime("%Y-%m-%dT%H-%M-%S"))
model_path = path.join('models', 'devel', 'crop_detector.h5')

callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, embeddings_freq=1),
    tf.keras.callbacks.ModelCheckpoint(model_path, save_best_only=True)]

model.compile(
    loss = 'sparse_categorical_crossentropy',
    optimizer = tf.keras.optimizers.Adam(),
    metrics   = ['acc'])

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 96, 96, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 96, 96, 16)        272       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 48, 48, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 32)        8224      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 18432)             0         
_________________________________________________________________
dense (Dense)                (None, 3)                 55299 

In [4]:
history = model.fit(
    train,
    steps_per_epoch=100,
    epochs=60,
    validation_data=validate,
    validation_steps=30,
    callbacks=callbacks)

Train for 100 steps, validate for 30 steps
Epoch 1/60
100/100 [==============================] - 9s 91ms/step - loss: 0.4541 - acc: 0.8388 - val_loss: 0.3863 - val_acc: 0.8271
Epoch 2/60
100/100 [==============================] - 6s 56ms/step - loss: 0.3321 - acc: 0.8681 - val_loss: 0.2932 - val_acc: 0.8854
Epoch 3/60
100/100 [==============================] - 5s 55ms/step - loss: 0.2868 - acc: 0.8881 - val_loss: 0.2458 - val_acc: 0.9104
Epoch 4/60
100/100 [==============================] - 5s 55ms/step - loss: 0.2395 - acc: 0.9044 - val_loss: 0.2294 - val_acc: 0.9083
Epoch 5/60
100/100 [==============================] - 5s 55ms/step - loss: 0.2087 - acc: 0.9150 - val_loss: 0.2058 - val_acc: 0.9250
Epoch 6/60
100/100 [==============================] - 6s 56ms/step - loss: 0.2065 - acc: 0.9194 - val_loss: 0.1906 - val_acc: 0.9240
Epoch 7/60
100/100 [==============================] - 6s 57ms/step - loss: 0.1812 - acc: 0.9294 - val_loss: 0.1692 - val_acc: 0.9354
Epoch 8/60
100/100 [======

In [5]:
print(model.evaluate(test, steps=100))

100/100 [==============================] - 9s 89ms/step - loss: 0.0905 - acc: 0.9691
[0.09050730416085571, 0.9690625]
